In [ ]:
torch.cuda.empty_cache()


In [1]:
%reload_ext autoreload
%autoreload 
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *
import scipy.io 
import numpy as np
import os
from tqdm import tqdm

In [1]:
# path = Path('.')
# path.ls()

## View the data

In [4]:
df=pd.read_csv(path/'train.csv')

In [15]:
df=df.drop('url',axis=1)
df.head()

,id,landmark_id
0,6e158a47eb2ca3f6,142820
1,202cd79556f30760,104169
2,3ad87684c99c06e1,37914
3,e7f70e9c61e66af3,102140
4,4072182eddd0100e,2474


In [ ]:
print(len(non_jpg),non_jpg[0:10])
print(results[0:10])
if '6e158a47eb2ca3f6' in results: print('exist')

In [ ]:
#LOAD different csv to list for review
import csv 

with open('fileNotExistTotal.csv','r') as f:
    reader = csv.reader(f)
    fnexistTotal=list(reader)

with open('idlist.csv','r') as f:
    reader = csv.reader(f)
    idlist=list(reader)
    
#delete unnecessary column head
for row in idlist: del row[0]
idlist=idlist[1:]
print(idlist[0:5])

with open('imglist.csv','r') as f:
    reader = csv.reader(f)
    results=list(reader)
for row in results: del row[0]
results=results[1:]
imglist=[]
for row in results:
    imglist.append(row[0])
print(imglist[0:5])

In [ ]:
#list of image id in train file

Idlist=[]
non_jpg=[]
pathTrain ='./train'
for f in tqdm(os.listdir(pathTrain)):
    if f.endswith('.jpg') :
        Idlist.append(os.path.splitext(f)[0])
#         old_file = os.path.join(pathTrain,f)
#         new_file = os.path.join(pathTrain,os.path.splitext(f)[0]+'.jpg')
#         os.rename(old_file,new_file)
    else:
        non_jpg.append(f)
print(len(results))

# Use multiprocessing to find missing images

In [ ]:
from multiprocessing import Pool

In [ ]:
def find_missing_trial(row):
    filexist=[]
    filenotexist=[]
    if row[1] in results:
        filexist = [row[1]]
    else:
        filenotexist = [row[1],row[2]]
    return filexist, filenotexist

In [ ]:
p = Pool()
matchlist = p.map(find_missing_trial, tqdm(Idlist),chunksize=500)

In [ ]:
#separate the list of missing file from multiprocessing result
fexist1=[]
fnexist1=[]
for row in matchlist:
#     print(row)
    if row[0] != []:
        fexist1.append(row[0])
    elif row[1] != []:
        fnexist1.append(row[1])
print('fneist' ,fnexist1)
p.close()
p.join()

In [ ]:
#confirm len of both list
len(fexist1)+len(fnexist1)

In [ ]:
#save finding missing img results
dfexist1=pd.DataFrame(fexist1)
print(dfexist1.head())
dfnexist1=pd.DataFrame(fnexist1)
print(dfnexist1.head())
dfexist1.to_csv(r'./fileExist1.csv', index=False, header=False)
dfnexist1.to_csv(r'./fileNotExist1.csv',index=False, header=False)

In [ ]:
dfexist.to_csv(r'./fileExist.csv')
dfnexist.to_csv(r'./fileNotExist.csv')

# step 1 - copy file from dataset at size 448 from another source

In [ ]:
#copy missing file to train256
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from shutil import copyfile

#look for missing file from train448
listmissingfrom448=[]
for i in tqdm(range(len(dfnexist1))):
    ID = dfnexist1['MissingID'][i]
    IDstr= str(ID+'.jpg')
    if os.path.exists('./train-448/'+IDstr):
        copyfile('./train-448/'+IDstr,'./train/'+IDstr)
    else:
        listmissingfrom448.append([ID,dfnexist1['url'][i]])

# step 2 - download image from url in train.csv

In [ ]:
#download missing file via URL link
import urllib3, shutil

urllist3=[]
for i in tqdm(range(len(listmissingfrom448))):
    if len(listmissingfrom448[i][1])>5:
        url=listmissingfrom448[i][1]
        c = urllib3.PoolManager()
        with c.request('GET',url, preload_content=False) as resp, open('./train/'+listmissingfrom448[i][0]+'.jpg', 'wb') as out_file:
            shutil.copyfileobj(resp, out_file)
    else:
        urllist3.append(listmissingfrom448[i])

In [ ]:
#look for missing file without url in train.csv
urlmissinglist=[]
for j in tqdm(range(len(urllist2))):
    for i in range(len(df)):
        if urllist2[j][0] in df['url'][i]:
            urlmissinglist.append([df['id'][i],df['url'][i]])
            break
# urllist[0][0]
# listmissingfrom448[122:135]

In [ ]:
dfurlmissing=pd.DataFrameFrame(urlmissinglist)
dfurlmissing.to_csv(r'./urlMissing.csv',index=False,header=False)

# step 3 - look through another source again and look through the dataset file on each urlmissingfile

In [ ]:
flist2=[]
for root,dirs,f in tqdm(os.walk('./google-landmark/')):
    for i in range(len(f)):
        if '0c2d084d5f3e50b8' in f[i]:
            flist2.append([root,dirs,f[i]])
flist2[0:100]

##  Find out landmark label with only one or two imgs 
## Separate the train & val set with imgs based on number of labelled imgs

In [18]:
df_test_split=df.copy()
print(df_test_split.head())
df.describe()

                 id  landmark_id
0  6e158a47eb2ca3f6       142820
1  202cd79556f30760       104169
2  3ad87684c99c06e1        37914
3  e7f70e9c61e66af3       102140
4  4072182eddd0100e         2474


,landmark_id
count,4.132914e+06
mean,1.014374e+05
std,5.870421e+04
min,0.000000e+00
25%,5.005400e+04
50%,1.016280e+05
75%,1.521240e+05
max,2.030930e+05


In [19]:
#count the number of repeated id
df_test_split['total'] = df_test_split.groupby('landmark_id')['landmark_id'].transform('count')

In [21]:
df_test_split.head()

,id,landmark_id,total
0,6e158a47eb2ca3f6,142820,21
1,202cd79556f30760,104169,208
2,3ad87684c99c06e1,37914,18
3,e7f70e9c61e66af3,102140,52
4,4072182eddd0100e,2474,439


In [22]:
df_grouped = df_test_split.groupby('landmark_id').apply(lambda x: x .sample(frac=0.2, random_state=47))

In [24]:
print(df_grouped.tail())
df_grouped.describe()

                                   id  landmark_id  total
landmark_id                                              
203092      3913115  3c82ce7d218544ce       203092     35
203093      2051838  534a3b1a1828a472       203093     20
            2637437  79e1699478893241       203093     20
            3811763  14bfae9178812ff6       203093     20
            570656   13b6f33751f5649f       203093     20


,landmark_id,total
count,822248.000000,822248.000000
mean,101429.200066,155.908492
std,58701.038810,584.732334
min,0.000000,3.000000
25%,50017.000000,20.000000
50%,101623.000000,51.000000
75%,152118.000000,125.000000
max,203093.000000,10247.000000


In [25]:
df_merged= pd.merge(left=df_test_split, right=df_grouped, on='id', how='left', suffixes = ('','_y'))
df_merged.head()

,id,landmark_id,total,landmark_id_y,total_y
0,6e158a47eb2ca3f6,142820,21,NaN,NaN
1,202cd79556f30760,104169,208,104169.0,208.0
2,3ad87684c99c06e1,37914,18,NaN,NaN
3,e7f70e9c61e66af3,102140,52,NaN,NaN
4,4072182eddd0100e,2474,439,NaN,NaN


In [26]:
df_merged['is_valid'] = df_merged.landmark_id_y.isnull()!=True
df_merged.head(20)

,id,landmark_id,total,landmark_id_y,total_y,is_valid
0,6e158a47eb2ca3f6,142820,21,NaN,NaN,False
1,202cd79556f30760,104169,208,104169.0,208.0,True
2,3ad87684c99c06e1,37914,18,NaN,NaN,False
3,e7f70e9c61e66af3,102140,52,NaN,NaN,False
4,4072182eddd0100e,2474,439,NaN,NaN,False
5,5554f8798114ed04,149463,41,NaN,NaN,False
6,6f31b874d1a4d489,6888,154,6888.0,154.0,True
7,16d8aa057cdd01b9,25719,11,NaN,NaN,False
8,3968e37e503f3109,122849,49,NaN,NaN,False
9,8df019949b8db328,81049,242,81049.0,242.0,True


In [28]:
df_merged.drop(['landmark_id_y','total_y'],axis=1,inplace=True)
df_merged.head()

,id,landmark_id,total,is_valid
0,6e158a47eb2ca3f6,142820,21,False
1,202cd79556f30760,104169,208,True
2,3ad87684c99c06e1,37914,18,False
3,e7f70e9c61e66af3,102140,52,False
4,4072182eddd0100e,2474,439,False


In [29]:
df_merged.to_csv('validation_random.csv', index=False)